### Country Vx Throughput Analysis
 
**Note:**

* Source:
  - covax_supply_chain_analytics.analysis_vx_throughput_data

* Target:
  - covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned

* Libraries: 
  - Python

* Built by: Jeremy Cooper
* Current owner: Jeremy Cooper
* Initial Build Date: 04/28/2021
* Latest Build Date: 04/28/2021

### Environment Management

In [0]:
# dbutils.widgets.removeAll()

In [0]:
# # Dataset Name, will be used for the Metastore Table, Folder Name for transformed outputs
# dbutils.widgets.text("Dataset", "dataset_name")

# # Project Name will be used for folder Name for transformed outputs
# dbutils.widgets.text("Project", "project_name")

# # Team name should be consistent with the Blob Storage Container
# dbutils.widgets.text("Partner","partner_name")

# # Team name should be consistent with the Blob Storage Container
# dbutils.widgets.text("Source","data_source")

# dbutils.widgets.text("iso_code", "")

#### Notebook Setup

##### Import any libraries or nested notebooks

In [0]:
from delta.tables import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.window import Window

##### Initialize File Paths

In [0]:
storage_root = "/mnt/"+dbutils.widgets.get("Partner")+"/"
storage_branch = "/"+dbutils.widgets.get("Source")+"/" +dbutils.widgets.get("Dataset")

raw_storage_path = storage_root + "raw" +storage_branch
dbfs_raw_storage_path = "/dbfs"+raw_storage_path

transformed_storage_path = storage_root + "transformed" +storage_branch
dbfs_transformed_storage_path = "/dbfs"+transformed_storage_path

print(raw_storage_path)
print(transformed_storage_path)

/mnt/covax-supply-chain-analytics/raw/who/analysis_vx_throughput_data_cleaned
/mnt/covax-supply-chain-analytics/transformed/who/analysis_vx_throughput_data_cleaned

### Get Data

In [0]:
iso_code = dbutils.widgets.get("iso_code")

In [0]:
# use this to get iso_code from country
iso_mapping = spark.sql("SELECT * FROM country_dimension.iso_mapping")

# get primary data
who = spark.sql("SELECT * FROM covax_supply_chain_analytics.analysis_vx_throughput_data")

### Transformation

In [0]:
# get iso code, clean additional country names
# convert date to date_week, based on "next" Friday date
df1 = who \
  .withColumn('total_doses', col('total_doses').cast(DoubleType())) \
  .withColumn('at_least_one_dose', col('at_least_one_dose').cast(DoubleType())) \
  .withColumn('fully_vaccinated', col('fully_vaccinated').cast(DoubleType())) \
  .withColumn('persons_booster_add_dose', col('persons_booster_add_dose').cast(DoubleType())) \
  .filter((col('total_doses').isNotNull()) & (col('total_doses')>0)) \
  .select('country_name', 'date', 'total_doses', 'at_least_one_dose', 'fully_vaccinated', 'persons_booster_add_dose', 'date_accessed') \
  .withColumn('date', to_date(col('date'))) \
  .withColumn('date_week', to_date(next_day(col('date'), 'Fri'))) \
  .drop_duplicates() \
  .join(iso_mapping, 'country_name', how='left') \
  .withColumn('iso_code', when(col('country_name')=='Bonaire, Sint Eustatius And Saba/Saba', 'BES1') \
              .when(col('country_name')=='Bonaire, Sint Eustatius And Saba/Sint Eustatius', 'BES1') \
              .when(col('country_name')=='Bonaire, Sint Eustatius And Saba', 'BES2') \
              .when(col('country_name')=='Bonaire, Sint Eustatius And Saba/Bonaire', 'XAA') \
              .otherwise(col('iso_code'))) \

# identify countries that have not reported for the latest week
max_date_week = df1.select(max(col('date_week')))
max_date_week = max_date_week.rdd.max()[0]

df1 = df1 \
  .withColumn('max_date_week', max('date_week').over(Window.partitionBy('iso_code'))) \
  .withColumn('is_latest_week_reported', when(col('max_date_week')==max_date_week, 1).otherwise(0)) \
  .drop('max_date_week')

print(max_date_week)
display(df1.filter(col('iso_code').isNull()))
display(df1.filter(col('iso_code')==iso_code))
display(df1.filter(col('is_latest_week_reported')==1))

2022-05-13

country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported


country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
China,2021-03-30,276600.0,276600.0,null,null,2022-05-09,2021-04-02,CHN,0
China,2021-07-16,5075829.0,2996060.0,2079769.0,null,2022-05-09,2021-07-23,CHN,0
China,2021-10-01,2.212881643E9,363651.0,311392.0,null,2022-05-09,2021-10-08,CHN,0
China,2022-04-18,3.31995E9,1.28E9,1.25E9,7.12169755E8,2022-05-09,2022-04-22,CHN,0
China,2021-12-24,963659.0,484212.0,452675.0,null,2022-05-09,2021-12-31,CHN,0
China,2021-05-28,2411055.0,1403017.0,1008038.0,null,2022-05-09,2021-06-04,CHN,0
China,2021-03-19,33144.0,31271.0,1873.0,null,2022-05-09,2021-03-26,CHN,0
China,2022-01-22,2.96437E9,null,null,null,2022-05-09,2022-01-28,CHN,0
China,2021-09-03,2.100462886E9,4508925.0,3779961.0,null,2022-05-09,2021-09-10,CHN,0
China,2022-01-28,1.2017519E7,1.275813554E9,1.235247554E9,956411.0,2022-05-09,2022-02-04,CHN,0


country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
Afghanistan,2021-10-10,2491952.0,846659.0,1645293.0,null,2022-05-09,2021-10-15,AFG,1
Afghanistan,2021-04-01,64697.0,null,null,null,2022-05-09,2021-04-02,AFG,1
Afghanistan,2021-08-01,1442250.0,763936.0,678314.0,null,2022-05-09,2021-08-06,AFG,1
Afghanistan,2022-02-07,5216998.0,4634282.0,3959887.0,null,2022-05-09,2022-02-11,AFG,1
Afghanistan,2022-01-27,5081064.0,4517380.0,3868832.0,null,2022-05-09,2022-01-28,AFG,1
Afghanistan,2021-12-23,4572319.0,4050948.0,3731333.0,null,2022-05-09,2021-12-24,AFG,1
Afghanistan,2022-02-23,5469463.0,4852237.0,4176422.0,null,2022-05-09,2022-02-25,AFG,1
Afghanistan,2021-08-09,1767239.0,769869.0,997370.0,null,2022-05-09,2021-08-13,AFG,1
Afghanistan,2021-05-24,573277.0,476367.0,96910.0,null,2022-05-09,2021-05-28,AFG,1
Afghanistan,2022-04-26,5985756.0,5286657.0,4636946.0,0.0,2022-05-09,2022-04-29,AFG,1


##### Total Doses Error Fix

In [0]:
# first round of error checking, identify the cases where total doses decreases, this is the master set of known data issues and will go into the data_errors tab in the output Excel
df_errors1 = df1 \
  .withColumn('total_doses_prev_period', lag(col('total_doses')).over(Window.partitionBy('iso_code').orderBy('date'))) \
  .filter(col('total_doses').cast(IntegerType()) < col('total_doses_prev_period').cast(IntegerType())) \
  .select('iso_code', 'country_name', 'date', 'total_doses', 'total_doses_prev_period', (col('total_doses') - col('total_doses_prev_period')).alias('total_doses_in_period')) \
  .withColumn('is_data_error', lit(1)) \

display(df_errors1.orderBy('iso_code', 'date'))
display(df_errors1.filter(col('iso_code')==iso_code).orderBy('iso_code', 'date'))

iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,is_data_error
AFG,Afghanistan,2021-08-20,1201286.0,1809517.0,-608231.0,1
AFG,Afghanistan,2022-04-04,5872684.0,5873352.0,-668.0,1
AGO,Angola,2022-05-01,1.7896626E7,1.8756893E7,-860267.0,1
AIA,Anguilla,2021-04-23,6035.0,6160.0,-125.0,1
ALB,Albania,2021-12-05,2035191.0,2035737.0,-546.0,1
ARE,United Arab Emirates,2021-08-12,1.7245815E7,1.7506683E7,-260868.0,1
ARM,Armenia,2021-07-18,132382.0,260813.0,-128431.0,1
ASM,American Samoa,2021-04-30,35936.0,37590.0,-1654.0,1
ATG,Antigua And Barbuda,2021-07-02,63750.0,63755.0,-5.0,1
AUS,Australia,2021-03-30,164437.0,295562.0,-131125.0,1


iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,is_data_error
CHN,China,2021-03-05,7719.0,1500000.0,-1492281.0,1
CHN,China,2021-03-19,33144.0,6.498E7,-6.4946856E7,1
CHN,China,2021-03-23,276600.0,7.496E7,-7.46834E7,1
CHN,China,2021-03-26,42210.0,276600.0,-234390.0,1
CHN,China,2021-03-30,276600.0,1.0242E8,-1.021434E8,1
CHN,China,2021-04-02,52903.0,1.10962E8,-1.10909097E8,1
CHN,China,2021-04-16,1057073.0,1.61861935E8,-1.60804862E8,1
CHN,China,2021-04-26,1196700.0,2.12290024E8,-2.11093324E8,1
CHN,China,2021-05-07,1737611.0,2.7905E8,-2.77312389E8,1
CHN,China,2021-05-28,2411055.0,5.46714E8,-5.44302945E8,1


In [0]:
display(df1.filter(col('iso_code')==iso_code).orderBy('iso_code', 'date'))
display(df1.select('iso_code').drop_duplicates().orderBy('iso_code'))

country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
China,2020-12-15,1500000.0,null,null,null,2022-05-09,2020-12-18,CHN,0
China,2021-03-05,7719.0,7700.0,19.0,null,2022-05-09,2021-03-12,CHN,0
China,2021-03-12,25732.0,24799.0,933.0,null,2022-05-09,2021-03-19,CHN,0
China,2021-03-14,6.498E7,null,null,null,2022-05-09,2021-03-19,CHN,0
China,2021-03-19,33144.0,31271.0,1873.0,null,2022-05-09,2021-03-26,CHN,0
China,2021-03-20,7.496E7,null,null,null,2022-05-09,2021-03-26,CHN,0
China,2021-03-23,276600.0,276600.0,null,null,2022-05-09,2021-03-26,CHN,0
China,2021-03-26,42210.0,35200.0,7010.0,null,2022-05-09,2021-04-02,CHN,0
China,2021-03-28,1.0242E8,null,null,null,2022-05-09,2021-04-02,CHN,0
China,2021-03-30,276600.0,276600.0,null,null,2022-05-09,2021-04-02,CHN,0


iso_code
ABW
AFG
AGO
AIA
ALB
AND
ARE
ARG
ARM
ASM


In [0]:
# list out countries for which we are manually fixing
manual_fix_list = ['AFG', 'AGO', 'AIA', 'ARE', 'ARM', 'ASM', 'ATG', 'AUS', 'AZE', 
                   'BDI', 'BEN', 'BFA', 'BGR', 'BHR', 'BLR', 'BRA', 'BTN', 'BWA', 
                   'CAN', 'CAF', 'CHN', 'COD', 'COK', 'COL', 'COM', 'CYM', 
                   'DMK', 'DMA', 'DJI',
                   'ESP', 'ETH',
                   'FRA', 'FSM',
                   'GAB', 'GBR', 'GEO', 'GLP', 'GMB', 'GNB', 'GNQ', 'GUM',
                   'HND', 'HUN', 
                   'IRL', 'ISL', 'ISR', 
                   'JOR', 'JPN', 
                   'KAZ', 'KGZ', 'KHM', 'KIR', 'KNA', 
                   'LSO', 'LUX', 'LVA', 
                   'MAR', 'MCO', 'MDV', 'MHL', 'MKD', 'MLT', 'MOZ', 'MYS', 
                   'NAM', 'NER', 'NIC', 'NIU', 'NPL', 'NZL',
                   'OMN',
                   'PAK', 'PHL', 'POL', 'PLW', 'PYF', 'PSE',
                   'QAT'
                   'ROU', 'RUS', 'RWA', 
                   'SAU', 'SDN', 'SEN', 'SLE', 'SSD', 'SVN', 'SYC', 
                   'TCD', 'TGO', 'TKM', 'TLS', 'TUR',
                   'UGA', 'UKR', 'USA', 'UZB', 
                   'VCT', 'VUT',
                   'WSM',
                   'XKX',
                   'YEM']
                  

# set na to 0
df1 = df1.fillna(0)

# fix data issues with total_doses
df2 = df1 \
  .filter(col('iso_code')!='BES1') \
  .withColumn('total_doses', when((col('iso_code')=='AFG') & (col('date')==('2021-08-20')), None) \
              .when((col('iso_code')=='AFG') & (col('date')==('2021-11-04')), None) \
              .when((col('iso_code')=='AFG') & (col('date')==('2022-04-03')), None) \
              .when((col('iso_code')=='AGO') & (col('date').between('2022-04-01', '2022-04-26')), None) \
              .when((col('iso_code')=='AIA') & (col('date')==('2021-04-23')), None) \
              .when((col('iso_code')=='ALB') & (col('date')==('2021-11-28')), None) \
              .when((col('iso_code')=='ARE') & (col('date')==('2021-08-11')), None) \
              .when((col('iso_code')=='ARM') & (col('date').between('2021-07-04', '2021-07-11')), None) \
              .when((col('iso_code')=='ASM') & (col('date').between('2021-04-30', '2021-05-05')), None) \
              .when((col('iso_code')=='ATG') & (col('date')==('2021-06-25')), None) \
              .when((col('iso_code')=='AUS') & (col('date')==('2021-03-30')), None) \
              .when((col('iso_code')=='AUS') & (col('date')==('2021-04-28')), None) \
              .when((col('iso_code')=='AUS') & (col('date')==('2021-08-14')), None) \
              .when((col('iso_code')=='AZE') & (col('date')==('2021-03-09')), None) \
              .when((col('iso_code')=='AZE') & (col('date')==('2021-04-04')), None) \
              .when((col('iso_code')=='BDI') & (col('date')==('2022-04-03')), None) \
              .when((col('iso_code')=='BEN') & (col('date')==('2021-05-31')), None) \
              .when((col('iso_code')=='BEN') & (col('date')==('2021-07-19')), None) \
              .when((col('iso_code')=='BEN') & (col('date').between('2021-06-10', '2021-06-24')), None) \
              .when((col('iso_code')=='BEN') & (col('date')==('2022-03-22')), None) \
              .when((col('iso_code')=='BEN') & (col('date').between('2022-04-17', '2022-05-01')), None) \
              .when((col('iso_code')=='BES1') & (col('date').between('2021-06-11', '2021-09-17')), None) \
              .when((col('iso_code')=='BES1') & (col('date').between('2021-10-08', '2021-11-12')), None) \
              .when((col('iso_code')=='BFA') & (col('date')==('2021-10-17')), None) \
              .when((col('iso_code')=='BFA') & (col('date')==('2021-07-19')), None) \
              .when((col('iso_code')=='BFA') & (col('date')==('2021-08-03')), None) \
              .when((col('iso_code')=='BGR') & (col('date')==('2021-07-25')), None) \
              .when((col('iso_code')=='BHR') & (col('date')==('2021-08-12')), None) \
              .when((col('iso_code')=='BLR') & (col('date')==('2022-03-20')), None) \
              .when((col('iso_code')=='BRA') & (col('date')==('2021-02-05')), None) \
              .when((col('iso_code')=='BRA') & (col('date')==('2022-02-18')), None) \
              .when((col('iso_code')=='BTN') & (col('date')==('2021-10-07')), None) \
              .when((col('iso_code')=='BTN') & (col('date')==('2021-10-14')), None) \
              .when((col('iso_code')=='BWA') & (col('date').between('2022-03-21', '2022-04-26')), None) \
              .when((col('iso_code')=='CAN') & (col('date')==('2021-10-01')), None) \
              .when((col('iso_code')=='CAN') & (col('date')==('2021-11-26')), None) \
              .when((col('iso_code')=='CAF') & (col('date')==('2021-09-01')), None) \
              .when((col('iso_code')=='CHN') & (col('date')==('2021-04-26')), None) \
              .when((col('iso_code')=='CHN') & (col('date')==('2021-05-25')), None) \
              .when((col('iso_code')=='CHN') & (col('date')==('2021-06-06')), None) \
              .when((col('iso_code')=='CHN') & (col('date')==('2021-08-14')), None) \
              .when((col('iso_code')=='COD') & (col('date')==('2022-03-24')), None) \
              .when((col('iso_code')=='COD') & (col('date').between('2022-04-24', '2022-05-08')), None) \
              .when((col('iso_code')=='COG') & (col('date')==('2021-11-06')), None) \
              .when((col('iso_code')=='COG') & (col('date')==('2022-05-01')), None) \
              .when((col('iso_code')=='COK') & (col('date')==('2022-02-06')), None) \
              .when((col('iso_code')=='COK') & (col('date')==('2022-03-29')), None) \
              .when((col('iso_code')=='COL') & (col('date')==('2021-04-23')), None) \
              .when((col('iso_code')=='COL') & (col('date')==('2021-05-28')), None) \
              .when((col('iso_code')=='COM') & (col('date')==('2021-07-19')), None) \
              .when((col('iso_code')=='CYM') & (col('date').between('2022-02-25', '2022-03-11')), None) \
              .when((col('iso_code')=='CYM') & (col('date')==('2022-04-01')), None) \
              .when((col('iso_code')=='DNK') & (col('date').between('2021-09-26', '2021-10-10')), None) \
              .when((col('iso_code')=='DNK') & (col('date').between('2022-01-02', '2022-01-09')), None) \
              .when((col('iso_code')=='DNK') & (col('date')==('2022-04-17')), None) \
              .when((col('iso_code')=='DMA') & (col('date')==('2021-04-16')), None) \
              .when((col('iso_code')=='DJI') & (col('date').between('2021-08-31', '2021-09-18')), None) \
              .when((col('iso_code')=='ETH') & (col('date')==('2022-04-10')), None) \
              .when((col('iso_code')=='ESP') & (col('date')==('2021-11-14')), None) \
              .when((col('iso_code')=='ESP') & (col('date')==('2022-01-30')), None) \
              .when((col('iso_code')=='FRA') & (col('date').between('2021-09-12', '2021-10-10')), None) \
              .when((col('iso_code')=='FSM') & (col('date').between('2021-04-30', '2021-05-05')), None) \
              .when((col('iso_code')=='GAB') & (col('date')==('2021-09-16')), None) \
              .when((col('iso_code')=='GAB') & (col('date')==('2022-01-22')), None) \
              .when((col('iso_code')=='GBR') & (col('date')==('2021-04-25')), None) \
              .when((col('iso_code')=='GBR') & (col('date')==('2021-05-08')), None) \
              .when((col('iso_code')=='GBR') & (col('date')==('2021-05-16')), None) \
              .when((col('iso_code')=='GEO') & (col('date')==('2021-03-28')), None) \
              .when((col('iso_code')=='GLP') & (col('date').between('2021-04-23', '2021-05-07')), None) \
              .when((col('iso_code')=='GLP') & (col('date').between('2022-02-11', '2022-02-18')), None) \
              .when((col('iso_code')=='GMB') & (col('date')==('2021-10-24')), None) \
              .when((col('iso_code')=='GMB') & (col('date')==('2021-11-05')), None) \
              .when((col('iso_code')=='GNB') & (col('date')==('2021-06-24')), None) \
              .when((col('iso_code')=='GNB') & (col('date')==('2021-08-20')), None) \
              .when((col('iso_code')=='GNB') & (col('date')==('2022-04-07')), None) \
              .when((col('iso_code')=='GNQ') & (col('date').between('2021-05-10', '2021-05-21')), None) \
              .when((col('iso_code')=='GNQ') & (col('date')==('2021-08-23')), None) \
              .when((col('iso_code')=='GUM') & (col('date').between('2021-05-07', '2021-05-18')), None) \
              .when((col('iso_code')=='GUY') & (col('date')==('2021-11-29')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2022-02-25')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2022-03-18')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2022-03-25')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2022-04-01')), None) \
              .when((col('iso_code')=='HUN') & (col('date')==('2021-06-06')), None) \
              .when((col('iso_code')=='HUN') & (col('date').between('2021-07-04', '2021-07-18')), None) \
              .when((col('iso_code')=='HUN') & (col('date')==('2021-08-08')), None) \
              .when((col('iso_code')=='IRL') & (col('date')==('2022-02-06')), None) \
              .when((col('iso_code')=='IRQ') & (col('date')==('2022-02-01')), None) \
              .when((col('iso_code')=='ISL') & (col('date').between('2021-06-27', '2021-07-11')), None) \
              .when((col('iso_code')=='ISL') & (col('date').between('2021-09-12', '2021-09-30')), None) \
              .when((col('iso_code')=='ISR') & (col('date')==('2021-11-07')), None) \
              .when((col('iso_code')=='ISR') & (col('date').between('2021-07-04', '2021-08-08')), None) \
              .when((col('iso_code')=='ISR') & (col('date').between('2021-12-12', '2021-12-26')), None) \
              .when((col('iso_code')=='ISR') & (col('date')==('2022-02-13')), None) \
              .when((col('iso_code')=='ISR') & (col('date')==('2022-03-06')), None) \
              .when((col('iso_code')=='JOR') & (col('date')==('2021-08-11')), None) \
              .when((col('iso_code')=='JOR') & (col('date')==('2021-10-27')), None) \
              .when((col('iso_code')=='JOR') & (col('date')==('2022-01-09')), None) \
              .when((col('iso_code')=='JOR') & (col('date')==('2022-04-11')), None) \
              .when((col('iso_code')=='JPN') & (col('date')==('2021-05-16')), None) \
              .when((col('iso_code')=='JPN') & (col('date')==('2022-01-03')), None) \
              .when((col('iso_code')=='KAZ') & (col('date').between('2021-05-02', '2021-05-09')), None) \
              .when((col('iso_code')=='KAZ') & (col('date').between('2021-08-22', '2021-08-29')), None) \
              .when((col('iso_code')=='KAZ') & (col('date')==('2021-09-12')), None) \
              .when((col('iso_code')=='KAZ') & (col('date')==('2021-12-12')), None) \
              .when((col('iso_code')=='KAZ') & (col('date').between('2022-02-06', '2022-02-20')), None) \
              .when((col('iso_code')=='KGZ') & (col('date')==('2021-05-02')), None) \
              .when((col('iso_code')=='KGZ') & (col('date')==('2021-05-30')), None) \
              .when((col('iso_code')=='KGZ') & (col('date')==('2021-08-22')), None) \
              .when((col('iso_code')=='KHM') & (col('date')==('2021-08-13')), None) \
              .when((col('iso_code')=='KHM') & (col('date').between('2021-10-22', '2021-10-24')), None) \
              .when((col('iso_code')=='KIR') & (col('date')==('2022-01-04')), None) \
              .when((col('iso_code')=='KNA') & (col('date')==('2022-02-11')), None) \
              .when((col('iso_code')=='KOR') & (col('date').between('2021-03-30', '2021-04-05')), None) \
              .when((col('iso_code')=='KOR') & (col('date')==('2021-10-17')), None) \
              .when((col('iso_code')=='KWT') & (col('date')==('2021-03-16')), None) \
              .when((col('iso_code')=='KWT') & (col('date')==('2021-04-28')), None) \
              .when((col('iso_code')=='KWT') & (col('date')==('2022-02-10')), None) \
              .when((col('iso_code')=='LBN') & (col('date')==('2021-08-31')), None) \
              .when((col('iso_code')=='LBR') & (col('date')==('2021-04-14')), None) \
              .when((col('iso_code')=='LBR') & (col('date')==('2021-10-08')), None) \
              .when((col('iso_code')=='LBR') & (col('date').between('2021-12-31', '2022-02-10')), None) \
              .when((col('iso_code')=='LSO') & (col('date').between('2021-05-21', '2021-05-24')), None) \
              .when((col('iso_code')=='LSO') & (col('date')==('2022-03-04')), None) \
              .when((col('iso_code')=='LUX') & (col('date').between('2021-07-18', '2021-07-25')), None) \
              .when((col('iso_code')=='LUX') & (col('date')==('2021-10-31')), None) \
              .when((col('iso_code')=='LUX') & (col('date')==('2022-02-13')), None) \
              .when((col('iso_code')=='LVA') & (col('date').between('2021-07-11', '2021-07-18')), None) \
              .when((col('iso_code')=='MAR') & (col('date')==('2022-04-26')), None) \
              .when((col('iso_code')=='MAR') & (col('date')==('2021-07-27')), None) \
              .when((col('iso_code')=='MCO') & (col('date')==('2021-07-04')), None) \
              .when((col('iso_code')=='MDV') & (col('date')==('2021-10-15')), None) \
              .when((col('iso_code')=='MHL') & (col('date')==('2021-05-07')), None) \
              .when((col('iso_code')=='MKD') & (col('date')==('2021-10-31')), None) \
              .when((col('iso_code')=='MLI') & (col('date').between('2022-04-17', '2022-04-26')), None) \
              .when((col('iso_code')=='MLI') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='MLT') & (col('date').between('2021-08-15', '2021-08-22')), None) \
              .when((col('iso_code')=='MLT') & (col('date')==('2021-09-27')), None) \
              .when((col('iso_code')=='MNE') & (col('date')==('2021-08-15')), None) \
              .when((col('iso_code')=='MNE') & (col('date')==('2022-02-27')), None) \
              .when((col('iso_code')=='MOZ') & (col('date')==('2022-04-09')), None) \
              .when((col('iso_code')=='MOZ') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='MRT') & (col('date')==('2021-11-14')), None) \
              .when((col('iso_code')=='MRT') & (col('date')==('2022-01-04')), None) \
              .when((col('iso_code')=='MRT') & (col('date')==('2022-04-06')), None) \
              .when((col('iso_code')=='MRT') & (col('date').between('2022-03-21', '2022-05-01')), None) \
              .when((col('iso_code')=='MUS') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='MYS') & (col('date')==('2021-09-20')), None) \
              .when((col('iso_code')=='NAM') & (col('date').between('2021-08-16', '2021-09-26')), None) \
              .when((col('iso_code')=='NAM') & (col('date').between('2021-10-11', '2021-11-13')), None) \
              .when((col('iso_code')=='NER') & (col('date').between('2021-07-05', '2021-07-13')), None) \
              .when((col('iso_code')=='NER') & (col('date')==('2021-08-28')), None) \
              .when((col('iso_code')=='NIC') & (col('date')==('2021-08-09')), None) \
#               .when((col('iso_code')=='NIC') & (col('date').between('2021-07-17', '2021-11-28')), None) \
              .when((col('iso_code')=='NIU') & (col('date')==('2021-07-16')), None) \
              .when((col('iso_code')=='NPL') & (col('date')==('2021-04-28')), None) \
              .when((col('iso_code')=='NZL') & (col('date')==('2021-08-09')), None) \
              .when((col('iso_code')=='OMN') & (col('date')==('2021-09-27')), None) \
              .when((col('iso_code')=='PAK') & (col('date')==('2021-11-14')), None) \
              .when((col('iso_code')=='PAK') & (col('date')==('2021-11-18')), None) \
              .when((col('iso_code')=='POL') & (col('date').between('2021-07-18', '2021-08-01')), None) \
              .when((col('iso_code')=='POL') & (col('date').between('2021-10-10', '2021-11-07')), None) \
              .when((col('iso_code')=='POL') & (col('date').between('2021-12-12', '2021-12-19')), None) \
              .when((col('iso_code')=='PHL') & (col('date')==('2021-07-26')), None) \
              .when((col('iso_code')=='PHL') & (col('date')==('2021-08-06')), None) \
              .when((col('iso_code')=='PLW') & (col('date')==('2021-05-05')), None) \
              .when((col('iso_code')=='PRI') & (col('date')==('2021-11-29')), None) \
              .when((col('iso_code')=='PRY') & (col('date')==('2022-02-25')), None) \
              .when((col('iso_code')=='PSE') & (col('date')==('2021-04-05')), None) \
              .when((col('iso_code')=='PSE') & (col('date')==('2021-03-02')), None) \
              .when((col('iso_code')=='PSE') & (col('date').between('2021-08-31', '2021-09-06')), None) \
              .when((col('iso_code')=='PSE') & (col('date').between('2021-10-18', '2021-10-25')), None) \
              .when((col('iso_code')=='PYF') & (col('date')==('2021-08-30')), None) \
              .when((col('iso_code')=='QAT') & (col('date')==('2021-08-11')), None) \
              .when((col('iso_code')=='QAT') & (col('date')==('2021-06-24')), None) \
              .when((col('iso_code')=='RUS') & (col('date')==('2021-05-09')), None) \
              .when((col('iso_code')=='RUS') & (col('date')==('2021-05-17')), None) \
              .when((col('iso_code')=='ROU') & (col('date')==('2021-06-27')), None) \
              .when((col('iso_code')=='ROU') & (col('date').between('2021-07-11', '2021-08-08')), None) \
              .when((col('iso_code')=='RWA') & (col('date').between('2021-04-28', '2021-06-17')), None) \
              .when((col('iso_code')=='RWA') & (col('date').between('2021-06-24', '2021-07-26')), None) \
              .when((col('iso_code')=='RWA') & (col('date')==('2021-08-23')), None) \
              .when((col('iso_code')=='RWA') & (col('date').between('2021-09-24', '2021-10-04')), None) \
              .when((col('iso_code')=='RWA') & (col('date')==('2021-10-18')), None) \
              .when((col('iso_code')=='RWA') & (col('date')==('2021-10-26')), None) \
              .when((col('iso_code')=='RWA') & (col('date')==('2021-11-10')), None) \
              .when((col('iso_code')=='SAU') & (col('date')==('2021-03-16')), None) \
              .when((col('iso_code')=='SAU') & (col('date')==('2021-07-11')), None) \
              .when((col('iso_code')=='SAU') & (col('date')==('2022-04-14')), None) \
              .when((col('iso_code')=='SDN') & (col('date').between('2021-04-28', '2021-05-09')), None) \
              .when((col('iso_code')=='SDN') & (col('date')==('2021-12-08')), None) \
              .when((col('iso_code')=='SEN') & (col('date').between('2021-06-14', '2021-06-17')), None) \
              .when((col('iso_code')=='SEN') & (col('date').between('2021-10-18', '2021-10-20')), None) \
              .when((col('iso_code')=='SEN') & (col('date').between('2021-12-09', '2021-12-21')), None) \
              .when((col('iso_code')=='SEN') & (col('date')==('2022-02-09')), None) \
              .when((col('iso_code')=='SEN') & (col('date')==('2022-03-01')), None) \
              .when((col('iso_code')=='SEN') & (col('date')==('2022-05-01')), None) \
              .when((col('iso_code')=='SEN') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='SGP') & (col('date').between('2021-10-02', '2021-10-22')), None) \
              .when((col('iso_code')=='SLE') & (col('date').between('2021-07-05', '2021-07-13')), None) \
              .when((col('iso_code')=='SOM') & (col('date')==('2021-04-21')), None) \
              .when((col('iso_code')=='SRB') & (col('date')==('2021-04-18')), None) \
              .when((col('iso_code')=='SSD') & (col('date')==('2022-01-01')), None) \
              .when((col('iso_code')=='STP') & (col('date').between('2022-03-07', '2022-05-01')), None) \
              .when((col('iso_code')=='SVN') & (col('date')==('2021-07-18')), None) \
              .when((col('iso_code')=='SVN') & (col('date').between('2022-02-13', '2022-02-20')), None) \
              .when((col('iso_code')=='SYC') & (col('date')==('2021-03-04')), None) \
              .when((col('iso_code')=='SYC') & (col('date')==('2021-08-25')), None) \
              .when((col('iso_code')=='SYC') & (col('date').between('2021-04-21', '2021-04-26')), None) \
              .when((col('iso_code')=='SYC') & (col('date')==('2021-07-10')), None) \
              .when((col('iso_code')=='SYC') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='SYR') & (col('date')==('2022-04-08')), None) \
              .when((col('iso_code')=='SYR') & (col('date')==('2022-04-26')), None) \
              .when((col('iso_code')=='TCD') & (col('date')==('2021-10-21')), None) \
              .when((col('iso_code')=='TGO') & (col('date')==('2021-05-10')), None) \
              .when((col('iso_code')=='TKM') & (col('date').between('2021-04-04', '2021-09-05')), None) \
              .when((col('iso_code')=='TLS') & (col('date')==('2021-10-02')), None) \
              .when((col('iso_code')=='TUN') & (col('date').between('2021-12-28', '2022-02-27')), None) \
              .when((col('iso_code')=='TUR') & (col('date')==('2021-05-09')), None) \
              .when((col('iso_code')=='TUR') & (col('date')==('2021-05-17')), None) \
              .when((col('iso_code')=='UGA') & (col('date')==('2021-05-18')), None) \
              .when((col('iso_code')=='UGA') & (col('date')==('2021-10-13')), None) \
              .when((col('iso_code')=='UKR') & (col('date')==('2021-04-11')), None) \
              .when((col('iso_code')=='UKR') & (col('date')==('2021-05-23')), None) \
              .when((col('iso_code')=='UKR') & (col('date').between('2021-02-27', '2021-03-02')), None) \
              .when((col('iso_code')=='USA') & (col('date').between('2021-11-26', '2021-11-29')), None) \
              .when((col('iso_code')=='UZB') & (col('date').between('2021-05-12', '2021-06-13')), None) \
              .when((col('iso_code')=='VCT') & (col('date').between('2021-07-23', '2021-08-09')), None) \
              .when((col('iso_code')=='VUT') & (col('date')==('2021-08-16')), None) \
              .when((col('iso_code')=='WSM') & (col('date')==('2021-06-25')), None) \
              .when((col('iso_code')=='XKX') & (col('date')==('2021-05-02')), None) \
              .when((col('iso_code')=='XKX') & (col('date')==('2021-05-30')), None) \
              .when((col('iso_code')=='XKX') & (col('date')==('2022-04-10')), None) \
              .when((col('iso_code')=='YEM') & (col('date').between('2021-11-14', '2022-01-17')), None) \
              .otherwise(col('total_doses'))) \
  .filter(col('total_doses').isNotNull())

# fix data issues with at_least_one_dose
df2 = df2 \
  .withColumn('at_least_one_dose',  when((col('iso_code')=='AFG') & (col('date').between('2021-07-14', '2021-07-27')), None) \
              .when((col('iso_code')=='AFG') & (col('date')==('2021-10-31')), None) \
              .when((col('iso_code')=='AGO') & (col('date')==('2021-08-23')), None) \
              .when((col('iso_code')=='AGO') & (col('date')==('2021-10-26')), None) \
              .when((col('iso_code')=='ALB') & (col('date')==('2021-04-24')), None) \
              .when((col('iso_code')=='ALB') & (col('date')==('2021-04-28')), None) \
              .when((col('iso_code')=='ALB') & (col('date')==('2021-05-04')), None) \
              .when((col('iso_code')=='AND') & (col('date').between('2021-03-19', '2021-04-12')), None) \
              .when((col('iso_code')=='AZE') & (col('date').between('2021-02-16', '2021-03-09')), None) \
              .when((col('iso_code')=='BFA') & (col('date').between('2021-10-17', '2021-10-31')), None) \
              .when((col('iso_code')=='BMU') & (col('date')==('2021-05-14')), None) \
              .when((col('iso_code')=='BMU') & (col('date')==('2021-06-25')), None) \
              .when((col('iso_code')=='DJI') & (col('date')==('2021-06-23')), None) \
              .when((col('iso_code')=='DJI') & (col('date')==('2021-08-17')), None) \
              .when((col('iso_code')=='COG') & (col('date')==('2021-04-14')), None) \
              .when((col('iso_code')=='COG') & (col('date')==('2021-07-26')), None) \
              .when((col('iso_code')=='COG') & (col('date').between('2021-06-08', '2021-06-10')), None) \
              .when((col('iso_code')=='COG') & (col('date').between('2021-11-15', '2021-11-16')), None) \
              .when((col('iso_code')=='COM') & (col('date')==('2021-05-31')), None) \
              .when((col('iso_code')=='COM') & (col('date').between('2021-08-20', '2021-09-06')), None) \
              .when((col('iso_code')=='DNK') & (col('date').between('2021-12-05', '2021-12-26')), None) \
              .when((col('iso_code')=='GMB') & (col('date')==('2021-09-02')), None) \
              .when((col('iso_code')=='HUN') & (col('date').between('2021-06-20', '2021-06-27')), None) \
              .when((col('iso_code')=='LBR') & (col('date').between('2021-08-23', '2021-09-03')), None) \
              .when((col('iso_code')=='STP') & (col('date')==('2021-07-26')), None) \
              .when((col('iso_code')=='ZAF') & (col('date')==('2021-06-20')), None) \
              .otherwise(col('at_least_one_dose'))) \

# fix data issues with fully_vaccinated
df2 = df2 \
  .withColumn('fully_vaccinated', when((col('iso_code')=='AFG') & (col('date')==('2021-08-30')), None) \
              .when((col('iso_code')=='AZE') & (col('date')==('2021-02-16')), None) \
              .when((col('iso_code')=='CAN') & (col('date')==('2021-04-23')), None) \
              .when((col('iso_code')=='COG') & (col('date').between('2021-07-12', '2021-08-03')), None) \
              .when((col('iso_code')=='DMA') & (col('date')==('2021-07-16')), None) \
              .when((col('iso_code')=='DNK') & (col('date').between('2021-12-05', '2021-12-26')), None) \
              .when((col('iso_code')=='DZA') & (col('date').between('2021-09-25', '2021-09-28')), None) \
              .when((col('iso_code')=='EGY') & (col('date')==('2021-09-11')), None) \
              .when((col('iso_code')=='EGY') & (col('date')==('2021-09-16')), None) \
              .when((col('iso_code')=='EGY') & (col('date').between('2021-05-09', '2021-05-11')), None) \
              .when((col('iso_code')=='GUY') & (col('date')==('2021-05-07')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2021-04-23')), None) \
              .when((col('iso_code')=='HND') & (col('date')==('2021-06-04')), None) \
              .when((col('iso_code')=='HND') & (col('date').between('2021-08-13', '2021-08-20')), None) \
              .when((col('iso_code')=='HND') & (col('date').between('2021-09-03', '2021-09-10')), None) \
              .when((col('iso_code')=='HUN') & (col('date')==('2021-08-01')), None) \
              .when((col('iso_code')=='PSE') & (col('date')==('2021-09-27')), None) \
              .when((col('iso_code')=='SEN') & (col('date')==('2022-05-08')), None) \
              .when((col('iso_code')=='STP') & (col('date').between('2021-07-05', '2021-07-13')), None) \
              .otherwise(col('fully_vaccinated'))) \


# second round of data issues, these have the manual fixes removed and "should be" one offs
df_errors2 = df2 \
  .withColumn('total_doses_prev_period', lag(col('total_doses')).over(Window.partitionBy('iso_code').orderBy('date'))) \
  .filter(col('total_doses').cast(IntegerType()) < col('total_doses_prev_period').cast(IntegerType())) \
  .select('iso_code', 'country_name', 'date', 'total_doses', 'total_doses_prev_period', (col('total_doses') - col('total_doses_prev_period')).alias('total_doses_in_period')) \
  .withColumn('to_remove', lit(1))

df1.cache()
df2.cache()
df_errors2.cache()

print(iso_code)
display(df1.filter(col('iso_code')==iso_code))
display(df2.filter(col('iso_code')==iso_code))

CHN

country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
China,2021-03-30,276600.0,276600.0,0.0,0.0,2022-05-09,2021-04-02,CHN,0
China,2021-05-28,2411055.0,1403017.0,1008038.0,0.0,2022-05-09,2021-06-04,CHN,0
China,2022-01-22,2.96437E9,0.0,0.0,0.0,2022-05-09,2022-01-28,CHN,0
China,2021-09-03,2.100462886E9,4508925.0,3779961.0,0.0,2022-05-09,2021-09-10,CHN,0
China,2021-04-20,1.9895E8,0.0,0.0,0.0,2022-05-09,2021-04-23,CHN,0
China,2021-04-26,1196700.0,789300.0,407400.0,0.0,2022-05-09,2021-04-30,CHN,0
China,2021-05-10,3.3964E8,0.0,0.0,0.0,2022-05-09,2021-05-14,CHN,0
China,2022-03-05,3.156899E9,0.0,0.0,0.0,2022-05-09,2022-03-11,CHN,0
China,2022-01-15,2.933485E9,0.0,0.0,0.0,2022-05-09,2022-01-21,CHN,0
China,2021-12-10,1.0536301E7,5277343.0,5007095.0,1.2058E8,2022-05-09,2021-12-17,CHN,0


country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
China,2021-03-30,276600.0,276600.0,0.0,0.0,2022-05-09,2021-04-02,CHN,0
China,2021-05-28,2411055.0,1403017.0,1008038.0,0.0,2022-05-09,2021-06-04,CHN,0
China,2022-01-22,2.96437E9,0.0,0.0,0.0,2022-05-09,2022-01-28,CHN,0
China,2021-09-03,2.100462886E9,4508925.0,3779961.0,0.0,2022-05-09,2021-09-10,CHN,0
China,2021-04-20,1.9895E8,0.0,0.0,0.0,2022-05-09,2021-04-23,CHN,0
China,2021-05-10,3.3964E8,0.0,0.0,0.0,2022-05-09,2021-05-14,CHN,0
China,2022-03-05,3.156899E9,0.0,0.0,0.0,2022-05-09,2022-03-11,CHN,0
China,2022-01-15,2.933485E9,0.0,0.0,0.0,2022-05-09,2022-01-21,CHN,0
China,2021-12-10,1.0536301E7,5277343.0,5007095.0,1.2058E8,2022-05-09,2021-12-17,CHN,0
China,2021-10-29,2.272044058E9,5058478.0,4759580.0,0.0,2022-05-09,2021-11-05,CHN,0


In [0]:
display(df2.filter(col('iso_code')==iso_code).orderBy('date'))

country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
China,2020-12-15,1500000.0,0.0,0.0,0.0,2022-05-09,2020-12-18,CHN,0
China,2021-03-05,7719.0,7700.0,19.0,0.0,2022-05-09,2021-03-12,CHN,0
China,2021-03-12,25732.0,24799.0,933.0,0.0,2022-05-09,2021-03-19,CHN,0
China,2021-03-14,6.498E7,0.0,0.0,0.0,2022-05-09,2021-03-19,CHN,0
China,2021-03-19,33144.0,31271.0,1873.0,0.0,2022-05-09,2021-03-26,CHN,0
China,2021-03-20,7.496E7,0.0,0.0,0.0,2022-05-09,2021-03-26,CHN,0
China,2021-03-23,276600.0,276600.0,0.0,0.0,2022-05-09,2021-03-26,CHN,0
China,2021-03-26,42210.0,35200.0,7010.0,0.0,2022-05-09,2021-04-02,CHN,0
China,2021-03-28,1.0242E8,0.0,0.0,0.0,2022-05-09,2021-04-02,CHN,0
China,2021-03-30,276600.0,276600.0,0.0,0.0,2022-05-09,2021-04-02,CHN,0


In [0]:
# check on what's still showing up as an error for total doses
df_errors1b = df2 \
  .withColumn('total_doses_prev_period', lag(col('total_doses')).over(Window.partitionBy('iso_code').orderBy('date'))) \
  .filter(col('total_doses').cast(IntegerType()) < col('total_doses_prev_period').cast(IntegerType())) \
  .select('iso_code', 'country_name', 'date', 'total_doses', 'total_doses_prev_period', (col('total_doses') - col('total_doses_prev_period')).alias('total_doses_in_period')) \
  .withColumn('is_data_error', lit(1)) \

display(df_errors1b.select('iso_code').drop_duplicates().orderBy('iso_code'))
display(df_errors1b.filter(col('iso_code')==iso_code).orderBy('iso_code', 'date'))
display(df_errors1b.orderBy('iso_code', 'date'))

iso_code
CHN


iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,is_data_error
CHN,China,2021-03-05,7719.0,1500000.0,-1492281.0,1
CHN,China,2021-03-19,33144.0,6.498E7,-6.4946856E7,1
CHN,China,2021-03-23,276600.0,7.496E7,-7.46834E7,1
CHN,China,2021-03-26,42210.0,276600.0,-234390.0,1
CHN,China,2021-03-30,276600.0,1.0242E8,-1.021434E8,1
CHN,China,2021-04-02,52903.0,1.10962E8,-1.10909097E8,1
CHN,China,2021-04-16,1057073.0,1.61861935E8,-1.60804862E8,1
CHN,China,2021-05-07,1737611.0,2.7905E8,-2.77312389E8,1
CHN,China,2021-05-28,2411055.0,5.27253E8,-5.24841945E8,1
CHN,China,2021-06-04,2692325.0,6.3172E8,-6.29027675E8,1


iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,is_data_error
CHN,China,2021-03-05,7719.0,1500000.0,-1492281.0,1
CHN,China,2021-03-19,33144.0,6.498E7,-6.4946856E7,1
CHN,China,2021-03-23,276600.0,7.496E7,-7.46834E7,1
CHN,China,2021-03-26,42210.0,276600.0,-234390.0,1
CHN,China,2021-03-30,276600.0,1.0242E8,-1.021434E8,1
CHN,China,2021-04-02,52903.0,1.10962E8,-1.10909097E8,1
CHN,China,2021-04-16,1057073.0,1.61861935E8,-1.60804862E8,1
CHN,China,2021-05-07,1737611.0,2.7905E8,-2.77312389E8,1
CHN,China,2021-05-28,2411055.0,5.27253E8,-5.24841945E8,1
CHN,China,2021-06-04,2692325.0,6.3172E8,-6.29027675E8,1


##### 1st Dose Error Fix

In [0]:
df_errors1st = df2 \
  .withColumn('1st_dose_prev_period', lag(col('at_least_one_dose')).over(Window.partitionBy('iso_code').orderBy('date'))) \
  .filter(col('at_least_one_dose').cast(IntegerType()) < col('1st_dose_prev_period').cast(IntegerType())) \
  .select('iso_code', 'country_name', 'date', 'at_least_one_dose', '1st_dose_prev_period', (col('at_least_one_dose') - col('1st_dose_prev_period')).alias('total_doses_in_period')) \
  .withColumn('to_remove_1st', lit(1)) \

display(df_errors1st.select('iso_code').drop_duplicates().orderBy('iso_code'))
display(df_errors1st.filter(col('iso_code')==iso_code).orderBy('iso_code', 'date'))
display(df_errors1st.orderBy('iso_code', 'date'))

iso_code
ARE
ATG
BDI
BEL
BFA
BHS
BLR
BMU
BRA
BTN


iso_code,country_name,date,at_least_one_dose,1st_dose_prev_period,total_doses_in_period,to_remove_1st


iso_code,country_name,date,at_least_one_dose,1st_dose_prev_period,total_doses_in_period,to_remove_1st
ARE,United Arab Emirates,2021-11-15,0.0,989050.0,-989050.0,1
ATG,Antigua And Barbuda,2021-08-06,38462.0,38592.0,-130.0,1
BDI,Burundi,2021-11-12,921.0,938.0,-17.0,1
BDI,Burundi,2021-11-15,871.0,921.0,-50.0,1
BDI,Burundi,2021-11-27,814.0,874.0,-60.0,1
BEL,Belgium,2021-07-18,7589653.0,7612710.0,-23057.0,1
BFA,Burkina Faso,2022-05-08,2432124.0,2434140.0,-2016.0,1
BHS,Bahamas,2021-10-29,132240.0,136110.0,-3870.0,1
BHS,Bahamas,2021-12-23,157041.0,157232.0,-191.0,1
BLR,Belarus,2022-02-06,5387508.0,5391526.0,-4018.0,1


##### 2nd Dose Error Fix

In [0]:
df_errors2nd = df2 \
  .withColumn('2nd_dose_prev_period', lag(col('fully_vaccinated')).over(Window.partitionBy('iso_code').orderBy('date'))) \
  .filter(col('fully_vaccinated').cast(IntegerType()) < col('2nd_dose_prev_period').cast(IntegerType())) \
  .select('iso_code', 'country_name', 'date', 'fully_vaccinated', '2nd_dose_prev_period', (col('fully_vaccinated') - col('2nd_dose_prev_period')).alias('total_doses_in_period')) \
  .withColumn('to_remove_2nd', lit(1)) \

display(df_errors2nd.select('iso_code').drop_duplicates().orderBy('iso_code'))
display(df_errors2nd.filter(col('iso_code')==iso_code).orderBy('iso_code', 'date'))
display(df_errors2nd.orderBy('iso_code', 'date'))

iso_code
AIA
ALB
ARE
BFA
BGR
BHR
BRA
BTN
CHE
CHN


iso_code,country_name,date,fully_vaccinated,2nd_dose_prev_period,total_doses_in_period,to_remove_2nd


iso_code,country_name,date,fully_vaccinated,2nd_dose_prev_period,total_doses_in_period,to_remove_2nd
AIA,Anguilla,2021-11-26,9156.0,9160.0,-4.0,1
ALB,Albania,2021-04-24,0.0,7071.0,-7071.0,1
ALB,Albania,2021-05-04,0.0,92065.0,-92065.0,1
ARE,United Arab Emirates,2021-10-27,0.0,8560000.0,-8560000.0,1
ARE,United Arab Emirates,2022-01-27,0.0,9347663.0,-9347663.0,1
BFA,Burkina Faso,2021-08-31,13273.0,13558.0,-285.0,1
BGR,Bulgaria,2022-02-13,2022585.0,2032960.0,-10375.0,1
BHR,Bahrain,2021-08-18,1073383.0,1267146.0,-193763.0,1
BRA,Brazil,2022-02-25,1.4877753E8,1.539E8,-5122470.0,1
BTN,Bhutan,2021-08-15,474547.0,475328.0,-781.0,1


##### Merge Together and Filter

In [0]:
# after applying the manual data fixes, this is the set of (in theory) one offs that we will remove from the dataset
# 2 and 3 below should return no records...if we are doing our job correctly

display(df_errors2)
display(df_errors2.filter(col('iso_code')==iso_code))
display(df_errors2.filter(col('iso_code').isin(manual_fix_list)))

iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,to_remove
CHN,China,2021-03-05,7719.0,1500000.0,-1492281.0,1
CHN,China,2021-03-19,33144.0,6.498E7,-6.4946856E7,1
CHN,China,2021-03-23,276600.0,7.496E7,-7.46834E7,1
CHN,China,2021-03-26,42210.0,276600.0,-234390.0,1
CHN,China,2021-03-30,276600.0,1.0242E8,-1.021434E8,1
CHN,China,2021-04-02,52903.0,1.10962E8,-1.10909097E8,1
CHN,China,2021-04-16,1057073.0,1.61861935E8,-1.60804862E8,1
CHN,China,2021-05-07,1737611.0,2.7905E8,-2.77312389E8,1
CHN,China,2021-05-28,2411055.0,5.27253E8,-5.24841945E8,1
CHN,China,2021-06-04,2692325.0,6.3172E8,-6.29027675E8,1


iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,to_remove


iso_code,country_name,date,total_doses,total_doses_prev_period,total_doses_in_period,to_remove
CHN,China,2021-03-05,7719.0,1500000.0,-1492281.0,1
CHN,China,2021-03-19,33144.0,6.498E7,-6.4946856E7,1
CHN,China,2021-03-23,276600.0,7.496E7,-7.46834E7,1
CHN,China,2021-03-26,42210.0,276600.0,-234390.0,1
CHN,China,2021-03-30,276600.0,1.0242E8,-1.021434E8,1
CHN,China,2021-04-02,52903.0,1.10962E8,-1.10909097E8,1
CHN,China,2021-04-16,1057073.0,1.61861935E8,-1.60804862E8,1
CHN,China,2021-05-07,1737611.0,2.7905E8,-2.77312389E8,1
CHN,China,2021-05-28,2411055.0,5.27253E8,-5.24841945E8,1
CHN,China,2021-06-04,2692325.0,6.3172E8,-6.29027675E8,1


In [0]:
df3 = df2 \
  .withColumn('manual_adjustment', when(col('iso_code').isin(manual_fix_list), 1).otherwise(0)) \
  .join(df_errors1.select('iso_code', 'date', 'is_data_error'), ['iso_code', 'date'], how='left') \
  .withColumn('is_data_error', when(col('is_data_error').isNull(), 0).otherwise(col('is_data_error'))) \
  .join(df_errors2.select('iso_code', 'date', 'to_remove'), ['iso_code', 'date'], how='left') \
  .withColumn('to_remove', when(col('to_remove').isNull(), 0).otherwise(col('to_remove'))) \
  .join(df_errors1st.select('iso_code', 'date', 'to_remove_1st'), ['iso_code', 'date'], how='left') \
  .withColumn('to_remove_1st', when(col('to_remove_1st').isNull(), 0).otherwise(col('to_remove_1st'))) \
  .join(df_errors2nd.select('iso_code', 'date', 'to_remove_2nd'), ['iso_code', 'date'], how='left') \
  .withColumn('to_remove_2nd', when(col('to_remove_2nd').isNull(), 0).otherwise(col('to_remove_2nd'))) \


display(df3.filter(col('iso_code')==iso_code))
# display(df3.filter((col('to_remove')==1) | (col('to_remove_1st')==1) | (col('to_remove_2nd')==1)))

iso_code,date,country_name,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,is_latest_week_reported,manual_adjustment,is_data_error,to_remove,to_remove_1st,to_remove_2nd
XKX,2021-10-03,Kosovo,1379793.0,812513.0,567280.0,0.0,2022-05-09,2021-10-08,0,1,0,0,0,0
XKX,2021-08-29,Kosovo,808078.0,538766.0,269312.0,0.0,2022-05-09,2021-09-03,0,1,0,0,0,0
XKX,2022-02-13,Kosovo,1804340.0,899560.0,810773.0,94007.0,2022-05-09,2022-02-18,0,1,0,0,0,0
XKX,2021-10-10,Kosovo,1456845.0,824278.0,632567.0,0.0,2022-05-09,2021-10-15,0,1,0,0,0,0
XKX,2021-09-26,Kosovo,1283838.0,778650.0,505188.0,0.0,2022-05-09,2021-10-01,0,1,0,0,0,0
XKX,2021-10-24,Kosovo,1542975.0,835679.0,707296.0,0.0,2022-05-09,2021-10-29,0,1,0,0,0,0
XKX,2021-05-20,Kosovo,55237.0,55237.0,0.0,0.0,2022-05-09,2021-05-21,0,1,0,0,0,0
XKX,2021-07-04,Kosovo,219788.0,177261.0,42527.0,0.0,2022-05-09,2021-07-09,0,1,0,0,0,0
XKX,2022-02-20,Kosovo,1808756.0,899560.0,813185.0,96011.0,2022-05-09,2022-02-25,0,1,0,0,0,0
XKX,2021-09-19,Kosovo,1176154.0,734532.0,441622.0,0.0,2022-05-09,2021-09-24,0,1,0,0,0,0


### Save to Azure Storage / Register in Databricks metastore

In [0]:
delta_path = transformed_storage_path + '.delta'

# dbutils.fs.rm(delta_path, True)

df3.write.format("delta").mode("overwrite").save(delta_path)

In [0]:
# path for delta
print(transformed_storage_path + '.delta')

/mnt/covax-supply-chain-analytics/transformed/who/analysis_vx_throughput_data_cleaned.delta

In [0]:
%sql

DROP TABLE IF EXISTS covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned;

CREATE TABLE covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned
USING DELTA
LOCATION '/mnt/covax-supply-chain-analytics/transformed/who/analysis_vx_throughput_data_cleaned.delta'

In [0]:
display(spark.sql("SELECT * FROM covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned") \
        .orderBy(['iso_code', 'date'])) \
#         .select('iso_code') \
#         .drop_duplicates()
#        )

iso_code,date,country_name,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,is_latest_week_reported,manual_adjustment,is_data_error,to_remove,to_remove_1st,to_remove_2nd
ABW,2021-02-19,Aruba,12.0,12.0,0.0,0.0,2022-05-09,2021-02-26,0,0,0,0,0,0
ABW,2021-02-26,Aruba,2896.0,2896.0,0.0,0.0,2022-05-09,2021-03-05,0,0,0,0,0,0
ABW,2021-03-05,Aruba,8882.0,8882.0,0.0,0.0,2022-05-09,2021-03-12,0,0,0,0,0,0
ABW,2021-03-12,Aruba,10679.0,10679.0,0.0,0.0,2022-05-09,2021-03-19,0,0,0,0,0,0
ABW,2021-03-19,Aruba,14628.0,11772.0,2856.0,0.0,2022-05-09,2021-03-26,0,0,0,0,0,0
ABW,2021-03-26,Aruba,21602.0,12835.0,8767.0,0.0,2022-05-09,2021-04-02,0,0,0,0,0,0
ABW,2021-04-01,Aruba,27904.0,17554.0,10350.0,0.0,2022-05-09,2021-04-02,0,0,0,0,0,0
ABW,2021-04-09,Aruba,35430.0,24102.0,11328.0,0.0,2022-05-09,2021-04-16,0,0,0,0,0,0
ABW,2021-04-16,Aruba,48254.0,36112.0,12142.0,0.0,2022-05-09,2021-04-23,0,0,0,0,0,0
ABW,2021-04-23,Aruba,62343.0,44421.0,17922.0,0.0,2022-05-09,2021-04-30,0,0,0,0,0,0


##### Query Delta Log

In [0]:
display(
  spark.sql("DESCRIBE HISTORY delta. `/mnt/covax-supply-chain-analytics/transformed/who/analysis_vx_throughput_data_cleaned.delta`")
)

version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata
72,2022-05-09T20:48:16.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,71,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12546, numOutputBytes -> 916528)",null
71,2022-05-09T17:17:19.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,70,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12574, numOutputBytes -> 917559)",null
70,2022-05-03T17:27:46.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,69,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12417, numOutputBytes -> 912728)",null
69,2022-05-03T17:02:59.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,68,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12424, numOutputBytes -> 912908)",null
68,2022-05-03T16:05:04.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,67,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12434, numOutputBytes -> 913326)",null
67,2022-04-26T03:18:12.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,66,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12050, numOutputBytes -> 902361)",null
66,2022-04-26T03:10:17.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,65,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 12052, numOutputBytes -> 902538)",null
65,2022-04-18T22:50:18.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,64,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 11937, numOutputBytes -> 898607)",null
64,2022-04-18T22:02:14.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,63,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 11944, numOutputBytes -> 898982)",null
63,2022-04-18T21:54:59.000+0000,6136552160696939,jeremy.cooper@gatesfoundation.org,WRITE,"Map(mode -> Overwrite, partitionBy -> [])",null,List(1884626790114436),1112-212424-shuwbub0,62,WriteSerializable,false,"Map(numFiles -> 133, numOutputRows -> 11946, numOutputBytes -> 899014)",null


### Appendix

In [0]:
df_changes = df1 \
  .select('iso_code', 'date', 'total_doses', 'at_least_one_dose', 'fully_vaccinated') \
  .join(spark.sql("SELECT * FROM covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned") \
          .select('iso_code', 'date', 'total_doses', 'at_least_one_dose', 'fully_vaccinated') \
          .withColumn('modified', lit(1)),
        ['iso_code', 'date', 'total_doses', 'at_least_one_dose', 'fully_vaccinated'], how='left') \
  .filter(col('modified').isNull())

print(df_changes.count())
display(df_changes.orderBy('iso_code', 'date'))

564

iso_code,date,total_doses,at_least_one_dose,fully_vaccinated,modified
AFG,2021-07-14,1024168.0,770229.0,253939.0,null
AFG,2021-07-18,1094257.0,811119.0,283138.0,null
AFG,2021-07-24,1171064.0,861336.0,309728.0,null
AFG,2021-07-27,1277085.0,939835.0,337250.0,null
AFG,2021-08-20,1201286.0,770542.0,430744.0,null
AFG,2021-08-30,1979652.0,773002.0,441371.0,null
AFG,2021-10-31,3398410.0,848598.0,2549812.0,null
AFG,2022-04-03,5873352.0,5187976.0,4532249.0,null
AGO,2021-08-23,1869933.0,1025212.0,844721.0,null
AGO,2021-10-26,6164219.0,4498272.0,1665947.0,null


In [0]:
display(df1.filter(col('iso_code')=='ABW'))
display(spark.sql("SELECT * FROM covax_supply_chain_analytics.analysis_vx_throughput_data_cleaned").filter(col('iso_code')=='ABW'))

country_name,date,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,iso_code,is_latest_week_reported
Aruba,2021-04-01,27904.0,17554.0,10350.0,0.0,2022-05-09,2021-04-02,ABW,0
Aruba,2021-10-08,157153.0,81694.0,75459.0,0.0,2022-05-09,2021-10-15,ABW,0
Aruba,2021-08-06,138724.0,72815.0,65909.0,0.0,2022-05-09,2021-08-13,ABW,0
Aruba,2021-05-07,78930.0,55143.0,23787.0,0.0,2022-05-09,2021-05-14,ABW,0
Aruba,2022-02-25,167759.0,87350.0,80409.0,0.0,2022-05-09,2022-03-04,ABW,0
Aruba,2021-10-22,158600.0,82256.0,76344.0,0.0,2022-05-09,2021-10-29,ABW,0
Aruba,2022-04-01,169341.0,87919.0,81422.0,0.0,2022-05-09,2022-04-08,ABW,0
Aruba,2022-04-08,169591.0,88013.0,81578.0,0.0,2022-05-09,2022-04-15,ABW,0
Aruba,2021-07-09,130116.0,68853.0,61263.0,0.0,2022-05-09,2021-07-16,ABW,0
Aruba,2021-12-17,162151.0,83857.0,78294.0,0.0,2022-05-09,2021-12-24,ABW,0


iso_code,date,country_name,total_doses,at_least_one_dose,fully_vaccinated,persons_booster_add_dose,date_accessed,date_week,is_latest_week_reported,manual_adjustment,is_data_error,to_remove,to_remove_1st,to_remove_2nd
ABW,2021-09-17,Aruba,153671.0,80436.0,73235.0,0.0,2022-05-09,2021-09-24,0,0,0,0,0,0
ABW,2021-10-29,Aruba,159230.0,82555.0,76675.0,0.0,2022-05-09,2021-11-05,0,0,0,0,0,0
ABW,2021-02-26,Aruba,2896.0,2896.0,0.0,0.0,2022-05-09,2021-03-05,0,0,0,0,0,0
ABW,2022-02-18,Aruba,167537.0,87229.0,80308.0,0.0,2022-05-09,2022-02-25,0,0,0,0,0,0
ABW,2021-07-23,Aruba,135015.0,71671.0,63344.0,0.0,2022-05-09,2021-07-30,0,0,0,0,0,0
ABW,2022-04-29,Aruba,170293.0,88370.0,81923.0,0.0,2022-05-09,2022-05-06,0,0,0,0,0,0
ABW,2022-03-18,Aruba,168495.0,87725.0,80770.0,0.0,2022-05-09,2022-03-25,0,0,0,0,0,0
ABW,2021-06-18,Aruba,122680.0,66466.0,56214.0,0.0,2022-05-09,2021-06-25,0,0,0,0,0,0
ABW,2022-04-08,Aruba,169591.0,88013.0,81578.0,0.0,2022-05-09,2022-04-15,0,0,0,0,0,0
ABW,2021-08-09,Aruba,139702.0,73050.0,66652.0,0.0,2022-05-09,2021-08-13,0,0,0,0,0,0
